# Final Project
## ADSP 32021 IP01 Machine Learning Operations
### 1: Data Pre-Processing
#### Group 2: Maria Clarissa Fionalita, Kajal Shukla, Mia Zhang, Priya Suvvaru Venkata

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "cpu"

In [ ]:
!python --version
!jupyter nbextension enable --py widgetsnbextension

Python 3.10.13
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
!git init
!dvc init -f
!dvc remote add -d storage gdrive://1wa6Ax2ro0D7EtdOYmmg0O7PmT8sfFGTN
!git commit .dvc/config -m "Configure remote storage"

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operations/Final Project/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guid

In [ ]:
!ls mashqa_data

test_webmd_squad_v2_consec.json   train_webmd_squad_v2_full.json
test_webmd_squad_v2_full.json	  val_webmd_squad_v2_consec.json
train_webmd_squad_v2_consec.json  val_webmd_squad_v2_full.json


In [ ]:
import json
from pathlib import Path
from pprint import pprint

# 1. Pre-Process Training Data

## Training Data V1
Only contains info from ```train_webmd_squad_v2_consec.json```

In [ ]:
file_path = "mashqa_data/train_webmd_squad_v2_consec.json"

data = json.loads(Path(file_path).read_text())
data = data["data"]

In [ ]:
context = {}

for index in range(len(data)):
    for p_index in range(len(data[index]["paragraphs"])):
        context_content = data[index]["paragraphs"][p_index]["context"]
        context[index] = [context_content]

In [ ]:
max_n_context = 0

for i in range(len(context)):
    if len(context[i]) > max_n_context:
        max_n_context = len(context[i])

pprint(max_n_context)

1


### Extract the context for inferences}

In [ ]:
# sample training data

pprint({'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'
})

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the '
            "Main Building's gold dome is a golden statue of the Virgin Mary. "
            'Immediately in front of the Main Building and facing it, is a '
            'copper statue of Christ with arms upraised with the legend '
            '"Venite Ad Me Omnes". Next to the Main Building is the Basilica '
            'of the Sacred Heart. Immediately behind the basilica is the '
            'Grotto, a Marian place of prayer and reflection. It is a replica '
            'of the grotto at Lourdes, France where the Virgin Mary reputedly '
            'appeared to Saint Bernadette Soubirous in 1858. At the end of the '
            'main drive (and in a direct line that connects through 3 statues '
            'and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did t

In [ ]:
training_data = []

for index in range(len(data)): # loop through data
    for p_index in range(len(data[index]["paragraphs"])): # loop through
        for q_index in range(len(data[index]["paragraphs"][p_index]["qas"])):
            data[index]["paragraphs"][p_index]["qas"][q_index]["context"] = context[index]
            training_data.append(data[index]["paragraphs"][p_index]["qas"][q_index])

In [ ]:
len(training_data)

19989

### Save and Push Training Data V1

In [ ]:
with open('data/training_data.json', 'w') as f:
    json.dump(training_data, f)

In [ ]:
# upload version 1 of the training data
!dvc add data/training_data.json
!git add data/.gitignore data/training_data.json.dvc

# push the v1 of the training data
!git commit -m "training data v1: pre-processed train_webmd_squad_v2_consec.json"
!cat data/training_data.json.dvc
!dvc push

                                                                core>⠋ Checking graph
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operation
                                                                                
!
  0%|          |Adding data/training_data.json to cach0/? [00:00<?,     ?file/s]
  0%|          |Adding data/training_data.json to cach0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/training_data.json to cach0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/? [00:00<?,    ?files/s]
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/1 [00:00<?,    ?files/s]
100%|██████████|Checking out /mnt/d/UChicago/Q4/A1/1 [00:00<00:00,  3.00files/s]
100% Adding...|

In [ ]:
!git log --oneline

bf4cb93 (HEAD -> master) training data v1: pre-processed train_webmd_squad_v2_consec.json
84b9203 Configure remote storage


## Training Data V2
Append info from ```train_webmd_squad_v2_full.json```

In [ ]:
file_path = "mashqa_data/train_webmd_squad_v2_full.json"

data = json.loads(Path(file_path).read_text())
data = data["data"]

In [ ]:
context = {}

for index in range(len(data)):
    for p_index in range(len(data[index]["paragraphs"])):
        context_content = data[index]["paragraphs"][p_index]["context"]
        context[index] = [context_content]

In [ ]:
len(context)

4532

In [ ]:
max_n_context = 0

for i in range(len(context)):
    if len(context[i]) > max_n_context:
        max_n_context = len(context[i])

pprint(max_n_context)

1


### Extract the context for inferences

In [ ]:
for index in range(len(data)): # loop through data
    for p_index in range(len(data[index]["paragraphs"])): # loop through
        for q_index in range(len(data[index]["paragraphs"][p_index]["qas"])):
            data[index]["paragraphs"][p_index]["qas"][q_index]["context"] = context[index]
            training_data.append(data[index]["paragraphs"][p_index]["qas"][q_index])

In [ ]:
len(training_data)

47717

### Save and Push Training Data V2

In [ ]:
with open('data/training_data.json', 'w') as f:
    json.dump(training_data, f)

In [ ]:
# upload version 2 of the training data
!dvc add data/training_data.json
!git add data/.gitignore data/training_data.json.dvc

# push the v2 of the training data
!git commit -m "training data v2: combine v1 with pre-processed train_webmd_squad_v2_full.json"
!cat data/training_data.json.dvc
!dvc push

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operation
                                                                                
!
  0%|          |Adding data/training_data.json to cach0/? [00:00<?,     ?file/s]
  0%|          |Adding data/training_data.json to cach0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/training_data.json to cach0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/? [00:00<?,    ?files/s]
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/1 [00:00<?,    ?files/s]
100%|██████████|Checking out /mnt/d/UChicago/Q4/A1/1 [00:00<00:00,  1.22files/s]
100% Adding...|█████████████████████████

In [ ]:
!git log --oneline

9c7cf0e (HEAD -> master) training data v2: combine v1 with pre-processed train_webmd_squad_v2_full.json
bf4cb93 training data v1: pre-processed train_webmd_squad_v2_consec.json
84b9203 Configure remote storage


### Checking that both v1 and v2 are properly stored

In [ ]:
# checkout v1
!git checkout bf4cb93 data/training_data.json.dvc
!dvc checkout

Updated 1 path from 4189675
Building workspace index                              |2.00 [00:00, 98.2entry/s]
Comparing indexes                                    |3.00 [00:00, 1.65kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
!
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Machi0.00/? [00:00<?,        ?B/s]
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Ma0.00/118M [00:00<?,        ?B/s]
 29%|██▉       |/mnt/d/UChicago/Q4/ADSP 32034.0M/118M [00:00<00:00,     347MB/s]
 62%|██████▏   |/mnt/d/UChicago/Q4/ADSP 32073.0M/118M [00:00<00:00,     379MB/s]
 93%|█████████▎|/mnt/d/UChicago/Q4/ADSP 3202110M/118M [00:00<00:00,     380MB/s]
Applying changes                                      |1.00 [00:00,  2.53file/s]
M       data/training_data.json


In [ ]:
training_data = json.loads(Path("data/training_data.json").read_text())
print(len(training_data))

19989


In [ ]:
# checkout v2
!git checkout 9c7cf0e data/training_data.json.dvc
!dvc checkout

Updated 1 path from e777d45
Building workspace index                              |2.00 [00:00,  110entry/s]
Comparing indexes                                    |3.00 [00:00, 1.59kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
!
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Machi0.00/? [00:00<?,        ?B/s]
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Ma0.00/279M [00:00<?,        ?B/s]
  9%|▊         |/mnt/d/UChicago/Q4/ADSP 32024.0M/279M [00:00<00:01,     245MB/s]
 21%|██        |/mnt/d/UChicago/Q4/ADSP 32059.0M/279M [00:00<00:00,     313MB/s]
 35%|███▌      |/mnt/d/UChicago/Q4/ADSP 32098.0M/279M [00:00<00:00,     356MB/s]
 49%|████▉     |/mnt/d/UChicago/Q4/ADSP 3202137M/279M [00:00<00:00,     375MB/s]
 63%|██████▎   |/mnt/d/UChicago/Q4/ADSP 3202175M/279M [00:00<00:00,     382MB/s]
 77%|███████▋  |/mnt/d/UChicago/Q4/ADSP 3202214M/279M [00:00<00:00,     388MB/s]
 91%|█████████ |/mnt/d/UChicago/Q4/ADSP 3202254M/279M [00:00<00:00,     396MB/s

In [ ]:
training_data = json.loads(Path("data/training_data.json").read_text())
print(len(training_data))

47717


# 2. Pre-Process Validation Data

## Validation Data V1
Only contains info from ```val_webmd_squad_v2_consec.json```

In [ ]:
file_path = "mashqa_data/val_webmd_squad_v2_consec.json"

data = json.loads(Path(file_path).read_text())
data = data["data"]

In [ ]:
context = {}

for index in range(len(data)):
    for p_index in range(len(data[index]["paragraphs"])):
        context_content = data[index]["paragraphs"][p_index]["context"]
        context[index] = [context_content]

In [ ]:
max_n_context = 0

for i in range(len(context)):
    if len(context[i]) > max_n_context:
        max_n_context = len(context[i])

pprint(max_n_context)

1


### Extract the context for inferences

In [ ]:
validation_data = []

for index in range(len(data)): # loop through data
    for p_index in range(len(data[index]["paragraphs"])): # loop through
        for q_index in range(len(data[index]["paragraphs"][p_index]["qas"])):
            data[index]["paragraphs"][p_index]["qas"][q_index]["context"] = context[index]
            validation_data.append(data[index]["paragraphs"][p_index]["qas"][q_index])

In [ ]:
len(validation_data)

2686

### Save and Push Validation Data V1

In [ ]:
with open('data/validation_data.json', 'w') as f:
    json.dump(validation_data, f)

In [ ]:
# upload version 1 of the validation data
!dvc add data/validation_data.json
!git add data/.gitignore data/validation_data.json.dvc

# push the v1 of the validation data
!git commit -m "validation data v1: mashqa_data/val_webmd_squad_v2_consec.json"
!cat data/validation_data.json.dvc
!dvc push

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operation
                                                                                
!
  0%|          |Adding data/validation_data.json to ca0/? [00:00<?,     ?file/s]
  0%|          |Adding data/validation_data.json to ca0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/validation_data.json to ca0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/? [00:00<?,    ?files/s]
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  3.17file/s]

To track the changes with git, run:

	g

In [ ]:
!git log --oneline

a1840fa (HEAD -> master) validation data v1: mashqa_data/val_webmd_squad_v2_consec.json
9c7cf0e training data v2: combine v1 with pre-processed train_webmd_squad_v2_full.json
bf4cb93 training data v1: pre-processed train_webmd_squad_v2_consec.json
84b9203 Configure remote storage


## Validation Data V2
Append info from ```val_webmd_squad_v2_full.json```

In [ ]:
file_path = "mashqa_data/val_webmd_squad_v2_full.json"

data = json.loads(Path(file_path).read_text())
data = data["data"]

In [ ]:
context = {}

for index in range(len(data)):
    for p_index in range(len(data[index]["paragraphs"])):
        context_content = data[index]["paragraphs"][p_index]["context"]
        context[index] = [context_content]

In [ ]:
max_n_context = 0

for i in range(len(context)):
    if len(context[i]) > max_n_context:
        max_n_context = len(context[i])

pprint(max_n_context)

1


### Extract the context for inferences

In [ ]:
for index in range(len(data)): # loop through data
    for p_index in range(len(data[index]["paragraphs"])): # loop through
        for q_index in range(len(data[index]["paragraphs"][p_index]["qas"])):
            data[index]["paragraphs"][p_index]["qas"][q_index]["context"] = context[index]
            validation_data.append(data[index]["paragraphs"][p_index]["qas"][q_index])

In [ ]:
len(validation_data)

6273

### Save and Push Validation Data V2

In [ ]:
with open('data/validation_data.json', 'w') as f:
    json.dump(validation_data, f)

In [ ]:
# upload version 2 of the validation data
!dvc add data/validation_data.json
!git add data/.gitignore data/validation_data.json.dvc

# push the v2 of the validation data
!git commit -m "validation data v2: mashqa_data/val_webmd_squad_v2_full.json"
!cat data/validation_data.json.dvc
!dvc push

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operation
                                                                                
!
  0%|          |Adding data/validation_data.json to ca0/? [00:00<?,     ?file/s]
  0%|          |Adding data/validation_data.json to ca0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/validation_data.json to ca0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/? [00:00<?,    ?files/s]
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/1 [00:00<?,    ?files/s]
100%|██████████|Checking out /mnt/d/UChicago/Q4/A1/1 [00:00<00:00,  7.37files/s]
100% Adding...|█████████████████████████

In [ ]:
!git log --oneline

bb55d3c (HEAD -> master) validation data v2: mashqa_data/val_webmd_squad_v2_full.json
a1840fa validation data v1: mashqa_data/val_webmd_squad_v2_consec.json
9c7cf0e training data v2: combine v1 with pre-processed train_webmd_squad_v2_full.json
bf4cb93 training data v1: pre-processed train_webmd_squad_v2_consec.json
84b9203 Configure remote storage


### Checking that both v1 and v2 are properly stored

In [ ]:
# checkout v1
!git checkout a1840fa data/validation_data.json.dvc
!dvc checkout

Updated 1 path from 88d64ca
Building workspace index                              |3.00 [00:00,  104entry/s]
Comparing indexes                                    |4.00 [00:00, 2.03kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
!
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Machi0.00/? [00:00<?,        ?B/s]
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 M0.00/18.0M [00:00<?,        ?B/s]
Applying changes                                      |1.00 [00:00,  10.4file/s]
M       data/validation_data.json
M       data/training_data.json


In [ ]:
validation_data = json.loads(Path("data/validation_data.json").read_text())
print(len(validation_data))

2686


In [ ]:
# checkout v2
!git checkout bb55d3c data/validation_data.json.dvc
!dvc checkout

Updated 1 path from c84db3e
Building workspace index                              |3.00 [00:00,  121entry/s]
Comparing indexes                                    |4.00 [00:00, 1.99kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
!
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Machi0.00/? [00:00<?,        ?B/s]
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 M0.00/41.5M [00:00<?,        ?B/s]
 72%|███████▏  |/mnt/d/UChicago/Q4/ADSP 3230.0M/41.5M [00:00<00:00,     314MB/s]
Applying changes                                      |1.00 [00:00,  6.07file/s]
M       data/training_data.json
M       data/validation_data.json


In [ ]:
validation_data = json.loads(Path("data/validation_data.json").read_text())
print(len(validation_data))

6273


# 3. Pre-Process Test Data

In [ ]:
!ls mashqa_data

test_webmd_squad_v2_consec.json   train_webmd_squad_v2_full.json
test_webmd_squad_v2_full.json	  val_webmd_squad_v2_consec.json
train_webmd_squad_v2_consec.json  val_webmd_squad_v2_full.json


## Test Data V1
Only contains info from ```test_webmd_squad_v2_consec.json```

In [ ]:
file_path = "mashqa_data/test_webmd_squad_v2_consec.json"

data = json.loads(Path(file_path).read_text())
data = data["data"]

In [ ]:
context = {}

for index in range(len(data)):
    for p_index in range(len(data[index]["paragraphs"])):
        context_content = data[index]["paragraphs"][p_index]["context"]
        context[index] = [context_content]

In [ ]:
max_n_context = 0

for i in range(len(context)):
    if len(context[i]) > max_n_context:
        max_n_context = len(context[i])

pprint(max_n_context)

1


### Extract the context for inferences

In [ ]:
test_data = []

for index in range(len(data)): # loop through data
    for p_index in range(len(data[index]["paragraphs"])): # loop through
        for q_index in range(len(data[index]["paragraphs"][p_index]["qas"])):
            data[index]["paragraphs"][p_index]["qas"][q_index]["context"] = context[index]
            test_data.append(data[index]["paragraphs"][p_index]["qas"][q_index])

In [ ]:
len(test_data)

2614

### Save and Push Test Data V1

In [ ]:
with open('data/test_data.json', 'w') as f:
    json.dump(test_data, f)

In [ ]:
# upload version 1 of the training data
!dvc add data/test_data.json
!git add data/.gitignore data/test_data.json.dvc

# push the v1 of the training data
!git commit -m "test data v1: pre-processed test_webmd_squad_v2_consec.json"
!cat data/test_data.json.dvc
!dvc push

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operation
                                                                                
!
  0%|          |Adding data/test_data.json to cache   0/? [00:00<?,     ?file/s]
  0%|          |Adding data/test_data.json to cache   0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/test_data.json to cache   0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/? [00:00<?,    ?files/s]
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  3.37file/s]

To track the changes with git, run:

	g

In [ ]:
!git log --oneline

9868399 (HEAD -> master) test data v1: pre-processed test_webmd_squad_v2_consec.json
bb55d3c validation data v2: mashqa_data/val_webmd_squad_v2_full.json
a1840fa validation data v1: mashqa_data/val_webmd_squad_v2_consec.json
9c7cf0e training data v2: combine v1 with pre-processed train_webmd_squad_v2_full.json
bf4cb93 training data v1: pre-processed train_webmd_squad_v2_consec.json
84b9203 Configure remote storage


## Test Data V2
Append info from ```train_webmd_squad_v2_full.json```

In [ ]:
file_path = "mashqa_data/test_webmd_squad_v2_full.json"

data = json.loads(Path(file_path).read_text())
data = data["data"]

In [ ]:
for index in range(len(data)):
    for p_index in range(len(data[index]["paragraphs"])):
        context_content = data[index]["paragraphs"][p_index]["context"]
        context[index] = [context_content]

In [ ]:
len(context)

521

In [ ]:
max_n_context = 0

for i in range(len(context)):
    if len(context[i]) > max_n_context:
        max_n_context = len(context[i])

pprint(max_n_context)

1


### Extract the context for inferences

In [ ]:
for index in range(len(data)): # loop through data
    for p_index in range(len(data[index]["paragraphs"])): # loop through
        for q_index in range(len(data[index]["paragraphs"][p_index]["qas"])):
            data[index]["paragraphs"][p_index]["qas"][q_index]["context"] = context[index]
            test_data.append(data[index]["paragraphs"][p_index]["qas"][q_index])

In [ ]:
len(test_data)

6107

### Save and Push Test Data V2

In [ ]:
with open('data/test_data.json', 'w') as f:
    json.dump(test_data, f)

In [ ]:
# upload version 2 of the test data
!dvc add data/test_data.json
!git add data/.gitignore data/test_data.json.dvc

# push the v2 of the test data
!git commit -m "test data v2: combine v1 with pre-processed test_webmd_squad_v2_full.json"
!cat data/test_data.json.dvc
!dvc push

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operation
                                                                                
!
  0%|          |Adding data/test_data.json to cache   0/? [00:00<?,     ?file/s]
  0%|          |Adding data/test_data.json to cache   0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/test_data.json to cache   0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/? [00:00<?,    ?files/s]
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/1 [00:00<?,    ?files/s]
100%|██████████|Checking out /mnt/d/UChicago/Q4/A1/1 [00:00<00:00,  7.50files/s]
100% Adding...|█████████████████████████

In [ ]:
!git log --oneline

7a85b8c (HEAD -> master) test data v2: combine v1 with pre-processed test_webmd_squad_v2_full.json
9868399 test data v1: pre-processed test_webmd_squad_v2_consec.json
bb55d3c validation data v2: mashqa_data/val_webmd_squad_v2_full.json
a1840fa validation data v1: mashqa_data/val_webmd_squad_v2_consec.json
9c7cf0e training data v2: combine v1 with pre-processed train_webmd_squad_v2_full.json
bf4cb93 training data v1: pre-processed train_webmd_squad_v2_consec.json
84b9203 Configure remote storage


### Checking that both v1 and v2 are properly stored

In [ ]:
# checkout v1
!git checkout 9868399 data/test_data.json.dvc
!dvc checkout

Updated 1 path from e983923
Building workspace index                              |4.00 [00:00,  122entry/s]
Comparing indexes                                    |5.00 [00:00, 2.53kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
!
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Machi0.00/? [00:00<?,        ?B/s]
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 M0.00/14.7M [00:00<?,        ?B/s]
Applying changes                                      |1.00 [00:00,  8.13file/s]
M       data/test_data.json
M       data/training_data.json
M       data/validation_data.json


In [ ]:
test_data = json.loads(Path("data/test_data.json").read_text())
print(len(test_data))

2614


In [ ]:
# checkout v2
!git checkout 7a85b8c data/test_data.json.dvc
!dvc checkout

Updated 1 path from c643473
Building workspace index                              |4.00 [00:00,  148entry/s]
Comparing indexes                                    |5.00 [00:00, 2.38kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
!
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Machi0.00/? [00:00<?,        ?B/s]
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 M0.00/34.7M [00:00<?,        ?B/s]
 95%|█████████▌|/mnt/d/UChicago/Q4/ADSP 3233.0M/34.7M [00:00<00:00,     340MB/s]
Applying changes                                      |1.00 [00:00,  7.12file/s]
M       data/training_data.json
M       data/validation_data.json
M       data/test_data.json


In [ ]:
test_data = json.loads(Path("data/test_data.json").read_text())
print(len(test_data))

6107


# 4. Pre-Process Full Data for Document Transformer

In [ ]:
!git log --oneline

7a85b8c (HEAD -> master) test data v2: combine v1 with pre-processed test_webmd_squad_v2_full.json
9868399 test data v1: pre-processed test_webmd_squad_v2_consec.json
bb55d3c validation data v2: mashqa_data/val_webmd_squad_v2_full.json
a1840fa validation data v1: mashqa_data/val_webmd_squad_v2_consec.json
9c7cf0e training data v2: combine v1 with pre-processed train_webmd_squad_v2_full.json
bf4cb93 training data v1: pre-processed train_webmd_squad_v2_consec.json
84b9203 Configure remote storage


## Full Data V1
Only contains info from ```*_consec.json```

In [ ]:
# checkout v1
!git checkout bf4cb93 data/training_data.json.dvc
!dvc checkout
!git checkout a1840fa data/validation_data.json.dvc
!dvc checkout
!git checkout 9868399 data/test_data.json.dvc
!dvc checkout

Updated 1 path from 4189675
Building workspace index                              |4.00 [00:00,  133entry/s]
Comparing indexes                                    |5.00 [00:00, 2.09kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
!
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Machi0.00/? [00:00<?,        ?B/s]
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Ma0.00/118M [00:00<?,        ?B/s]
 25%|██▌       |/mnt/d/UChicago/Q4/ADSP 32030.0M/118M [00:00<00:00,     309MB/s]
 55%|█████▌    |/mnt/d/UChicago/Q4/ADSP 32065.0M/118M [00:00<00:00,     341MB/s]
 83%|████████▎ |/mnt/d/UChicago/Q4/ADSP 32098.0M/118M [00:00<00:00,     342MB/s]
Applying changes                                      |1.00 [00:00,  2.32file/s]
M       data/test_data.json
M       data/validation_data.json
M       data/training_data.json
Updated 1 path from 88d64ca
Building workspace index                              |4.00 [00:00,  130entry/s]
Comparing indexes                     

In [ ]:
full_data = json.loads(Path("data/training_data.json").read_text()) + json.loads(Path("data/validation_data.json").read_text()) +\
json.loads(Path("data/test_data.json").read_text())

pprint(len(full_data))

25289


### Save and Push Full Data V1

In [ ]:
with open('data/full_data.json', 'w') as f:
    json.dump(full_data, f)

In [ ]:
# upload version 1 of the full data
!dvc add data/full_data.json
!git add data/.gitignore data/full_data.json.dvc

# push the v1 of the full data
!git commit -m "full data v1: *_consec.json"
!cat data/full_data.json.dvc
!dvc push

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operation
                                                                                
!
  0%|          |Adding data/full_data.json to cache   0/? [00:00<?,     ?file/s]
  0%|          |Adding data/full_data.json to cache   0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/full_data.json to cache   0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/? [00:00<?,    ?files/s]
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/1 [00:00<?,    ?files/s]
100%|██████████|Checking out /mnt/d/UChicago/Q4/A1/1 [00:00<00:00,  2.23files/s]
100% Adding...|█████████████████████████

# Full Data V2
Contains all ```*.json```

In [ ]:
# checkout v1
!git checkout 9c7cf0e data/training_data.json.dvc
!dvc checkout
!git checkout bb55d3c data/validation_data.json.dvc
!dvc checkout
!git checkout 7a85b8c data/test_data.json.dvc
!dvc checkout

Updated 1 path from e777d45
Building workspace index                              |5.00 [00:00,  138entry/s]
Comparing indexes                                    |6.00 [00:00, 2.65kentry/s]
Applying changes                                      |0.00 [00:00,     ?file/s]
!
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Machi0.00/? [00:00<?,        ?B/s]
  0%|          |/mnt/d/UChicago/Q4/ADSP 32021 Ma0.00/279M [00:00<?,        ?B/s]
 11%|█▏        |/mnt/d/UChicago/Q4/ADSP 32032.0M/279M [00:00<00:00,     335MB/s]
 24%|██▍       |/mnt/d/UChicago/Q4/ADSP 32068.0M/279M [00:00<00:00,     359MB/s]
 38%|███▊      |/mnt/d/UChicago/Q4/ADSP 3202107M/279M [00:00<00:00,     380MB/s]
 51%|█████     |/mnt/d/UChicago/Q4/ADSP 3202141M/279M [00:00<00:00,     367MB/s]
 63%|██████▎   |/mnt/d/UChicago/Q4/ADSP 3202176M/279M [00:00<00:00,     367MB/s]
 76%|███████▌  |/mnt/d/UChicago/Q4/ADSP 3202211M/279M [00:00<00:00,     365MB/s]
 86%|████████▋ |/mnt/d/UChicago/Q4/ADSP 3202241M/279M [00:00<00:00,     347MB/s

In [ ]:
full_data = json.loads(Path("data/training_data.json").read_text()) + json.loads(Path("data/validation_data.json").read_text()) +\
json.loads(Path("data/test_data.json").read_text())

pprint(len(full_data))

60097


### Save and Push Full Data V2

In [ ]:
with open('data/full_data.json', 'w') as f:
    json.dump(full_data, f)

In [ ]:
# upload version 2 of the full data
!dvc add data/full_data.json
!git add data/.gitignore data/full_data.json.dvc

# push the v2 of the full data
!git commit -m "full data v2: *.json"
!cat data/full_data.json.dvc
!dvc push

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/mnt/d/UChicago/Q4/ADSP 32021 Machine Learning Operation
                                                                                
!
  0%|          |Adding data/full_data.json to cache   0/? [00:00<?,     ?file/s]
  0%|          |Adding data/full_data.json to cache   0/1 [00:00<?,     ?file/s]
  0%|          |Adding data/full_data.json to cache   0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/? [00:00<?,    ?files/s]
  0%|          |Checking out /mnt/d/UChicago/Q4/ADSP 30/1 [00:00<?,    ?files/s]
100%|██████████|Checking out /mnt/d/UChicago/Q4/A1/1 [00:00<00:00,  1.01files/s]
100% Adding...|█████████████████████████